In [ ]:
# all imports
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import os
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
import random
import torch
from torch import nn, optim
import math
from IPython import display
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import pdb
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn import metrics
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error


In [ ]:
# upload train df
traindf = pd.read_csv("train.csv")

In [ ]:
# create dictionary of # of instances for each acne type
double_array = [[" blackheads", 230], [" dark spot", 605], [" nodules", 147], [" papules", 645], [" pustules", 361], [" whiteheads", 317]]
classCount = {item[0]: item[1] for item in double_array}

In [ ]:
# iterate through all classes
for a_type in classCount:
  # depending on count, over sample more or le
  count = classCount[a_type]
  if count < 300:
    count_to_change = int((0.18 * count) / 3)
  else:
    count_to_change = int((0.05 * count) / 3)

  change_im_df = traindf[traindf[a_type] == 1].sample(n=count_to_change)

  count = 0
  for idx, row in change_im_df.iterrows():
    count += 1
    # open original image
    image_path = row['filename']
    image = Image.open(image_path)


    # apply shear transformation
    sheared_im = image.transform(image.size, Image.AFFINE, (1, 0.2, 0, 0.3, 1, 0), fillcolor='black')
    # save altered image
    sheared_filename = os.path.join(f"new_sheared_{count}.png")
    sheared_im.save(sheared_filename)
    #duplicate row (have same blackhead, whitehead etc properties) but change the filename to the altered image path
    df_new_shear = traindf.loc[traindf['filename'] == image_path].copy()
    df_new_shear['filename'] = sheared_filename
    # add back into traindf
    traindf = pd.concat([traindf, df_new_shear], ignore_index=True)


    # apply resize (pixelate)
    pixelated_im = image.resize((300, 226))
    # save altered image
    pixelated_filename = os.path.join(f"new_pixelated_{count}.png")
    pixelated_im.save(pixelated_filename)
    # duplicate row (have same blackhead, whitehead etc properties) but change the filename to the altered image path
    df_new_pixel = traindf.loc[traindf['filename'] == image_path].copy()
    df_new_pixel['filename'] = pixelated_filename
    # add back into traindf
    traindf = pd.concat([traindf, df_new_pixel], ignore_index=True)

    # apply brightness reduction
    enhancer = ImageEnhance.Brightness(image)
    darker_im = enhancer.enhance(0.5)
    # save altered image
    darker_filename = os.path.join(f"new_darker_{count}.png")
    darker_im.save(darker_filename)
    # duplicate row (have same blackhead, whitehead etc properties) but change the filename to the altered image path
    df_new_darker = traindf.loc[traindf['filename'] == image_path].copy()
    df_new_darker['filename'] = darker_filename
    # add back into traindf
    traindf = pd.concat([traindf, df_new_darker], ignore_index=True)